In [1]:
cd ~/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/

/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files


In [2]:
# system/os/regex and basic math functions
import sys
import os
import re
import math
import pickle
import dateutil
import datetime as dt
from itertools import permutations as npn
from itertools import chain
from functools import partial
from fuzzywuzzy.process import extractBests, extract, extractOne
 
# IPython display convenience stuff
from IPython.display import display_html, display_javascript
from IPython import __version__ as ipythonversion
HTML = partial(display_html, raw=True)
print("IPython: {}".format(ipythonversion))
    
# Set logging level
import logging
logging.getLogger("").setLevel(logging.INFO)
 
# numpy for matrix algebra
import numpy as np
print("Numpy: {}".format(np.version.full_version))
 
# scipy for probability distributions and some statistical tests
import scipy as sp
import scipy.stats as stats
print("Scipy: {}".format(sp.version.full_version))
 
# pandas for data manipulation
import pandas as pd
print("Pandas: {}".format(pd.version.version))
 
# Set pandas display options for pretty pretty printing
pd.set_option('html', True, 'precision', 4)
pd.set_option('max_rows',200,'max_columns',50)
 
# pandas web interface for things like FF factors
import pandas.io.data as web
 
# pytables for hdf5 library (on-disk storage)
# import tables as tb
 
# SQLAlchemy for relational db management
import sqlalchemy as sa
print("SQLalchemy: {}".format(sa.__version__))

# matplotlib for plotting and pyplot for MATLAB-style API
import matplotlib as mpl
import matplotlib.pyplot as plt
print("MatPlotLib: {}".format(mpl.__version__))
 
# display plots inline
%matplotlib inline
 
# statsmodels for models with a formula framework similar to R 
import statsmodels.api as sm
print("Statsmodels: {}".format(sm.version.full_version))

IPython: 3.2.0
Numpy: 1.9.2
Scipy: 0.15.1
Pandas: 0.16.2
SQLalchemy: 1.0.5
MatPlotLib: 1.4.3
Statsmodels: 0.6.1


In [3]:
debug = False
plots = True
KATRINA_DATE = dt.datetime(2005, 8, 23)
KATRINA_QUARTER = 3

Column headings from KBD_emaxx_cleaning.ipynb, but a subset that I care about. Not all.

In [4]:
def add_column(df, name, func):
    """
    Adds a column to *df* named *name* which is created by:
    df[name] = df.apply(func, axis=1)
    """
    df[name] = df.apply(func, axis=1)
    return df

# Load small files into pandas and search for fund names

In [5]:
!head ../../compustat_cusips.tab -n 2

PERMNO	PERMCO	COMNAM	TICKER	cusip	cusip6	gvkey	CIK	sic	date_start	date_end	year	quarter	date	fic	comp
76868	10817	AAON INC	AAON	00036020	000360				2012-05-02	2013-12-31				USA	1


In [6]:
!head ISSUERS.TXT -n 2

cusip6,year,quarter,name,name_sort,country,ticker
00023*,2000,1,AAA COOPER TRANSPORTATION,AAA COOPER TRANSPORTATION,USA,


In [7]:
col_str = """
ISSUERS.TXT: cusip6, year, quarter, name, name_sort, country, ticker
SECMAST.TXT: cusip6, cusip78, year, quarter, issue_desc, coupon_rate, coupon_structure, currency_code, maturity_year, maturity_date, market_sector, collateral_code, cusip_source, private, issue_amt_outstanding, issue_amt_change, num_held, num_buying, num_selling, total_par_held, pledge_code, issue_date, moody, s_and_p, fitch, duff_and_phelps
FUND.TXT: sub_account_id, year, quarter, sub_account_class, sub_account_name, sub_account_name_sort, managing_firm_id, sub_account_country, total_par_amt_held, total_number_bonds_held, update_date
HOLDING.TXT: cusip6, cusip78, sub_account_id, par_amount, report_date, par_amount_change, managing_firm_id, acquisition_book_value, year, quarter
FIRM.TXT: managing_firm_id, year, quarter, managing_firm_name, managing_firm_name_sort, managing_firm_type, managing_firm_country_code, managing_firm_total_par_held, managing_firm_number_issues_held
""".strip().split('\n')

column_headings = {ln.split(': ')[0]:ln.split(': ')[1].split(', ') for ln in col_str}

ROOTDIR = '/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/'

In [8]:
"""
Load all eMAXX files into pandas dataframes except HOLDING which is huge.
"""
dfs = {'firmtype':pd.read_csv('2002_Q2/Q2/FIRMTYPE.TXT', header=None, 
                              names=['managing_firm_type_name', 'managing_firm_type']),
       'fundtype':pd.read_csv('2002_Q2/Q2/FUNDCLAS.TXT', header=None, 
                              names=['sub_account_class_name', 'sub_account_class']),
       'compcusip':pd.read_csv(os.path.join(ROOTDIR, 'compustat_cusips.tab'), sep='\t')}
if debug: print()
for fname,cols in column_headings.items():
    if fname in ('HOLDING.TXT',): continue
    with open(fname) as fh:
        tmp_df = pd.read_csv(fh, usecols=cols, low_memory=False,
                                 true_values=('Y',), false_values=('N',))
        dfs[fname.split('.')[0].lower()] = tmp_df
        if debug:
            print(fname)
            print(str(dfs[fname.split('.')[0].lower()].dtypes))
            print()

In [9]:
def mkre(*args):
    """
    Make regular expression from list of words.
    Ex:
        in: a, b
        out: (?:a\\s*b|b\\s*a)
    """
    return re.compile("(?:{})".format(".*".join(args)), re.I)
    #return re.compile("(?:{})".format("|".join([".*".join(x) for x in npn(args,len(args))])), re.I)
mkre('a','b')

re.compile(r'(?:a.*b)', re.IGNORECASE|re.UNICODE)

# Search for Managing Firms

Uses regular expressions to allow for word order differences and odd hyphenation and whatnot.
Outputs managing_ids into mids list, for use in filtering dataframes.

In [10]:
firm_res = {
"State Farm Insurance Company": mkre('state','farm'),
"Allstate Insurance Co Group": mkre('allstate','insurance'),
"Progressive Casualty Group": mkre('progressive','capital'),
"Alfa Insurance": mkre('alfa','insurance'),
"Mississippi Farm Bureau Casualty Insurance": mkre('farm','bureau','casualty'),
"St. Paul Travelers Companies": mkre('st','paul','travelers'),
"Endurance Reinsurance Corp of America": mkre('endurance','reinsurance'),
"Olympus Insurance": mkre('olympus','insurance'),
"Partner Reinsurance United States": mkre('partner','reinsurance'),
"United Services Automobile Association (usaa)": mkre('usaa'),
#
# below here match sub_account_ids
"American Modern Home Insurance": mkre('american','modern', 'home'),
"American International Insurance": mkre('american','international','insurance','co'),
"Ace American Reinsurance": mkre('ace','american','reinsurance'),
"Alea North America Insurance": mkre('alea','north','america'),
"Nationwide Assurance": mkre('nationwide','assurance'),
"Odyssey America Reinsurance": mkre('odyssey','america'),
"Transatlantic Reinsurance United States": mkre('transatlantic','reinsurance'),
}
force_ids = {'mids':{}, 
             'saids':{'American International Insurance':[26386]}}

In [11]:
"""
Searches managing_firm_name for firm RegularExpressions 
and puts the IDs in a dictionary (and /mids/ list)
"""
#print(mids, '<- old')
managing_firm_ids = {}
for firm_name, firm_re in firm_res.items():
    if firm_name in force_ids['mids']:
        managing_firm_ids[firm_name] = force_ids['mids'][firm_name]
        continue
    managing_firm_ids[firm_name] = list(dfs['firm'][
            dfs['firm'].apply(lambda x: bool(firm_re.search(x.managing_firm_name)), axis=1)
        ].managing_firm_id.unique())
mids = list(map(int, chain(*managing_firm_ids.values())))
print(mids)
#pd.merge(left=managing_firms, right=dfs['firmtype'], how='left').drop_duplicates()

[12284, 12471, 35287, 12834, 10426, 16655, 27570, 10022, 10032, 11950, 27633]


In [12]:
"""
Searches sub_account_name for firm RegularExpressions for 
the firms that don't match on managing_firm name
and puts the IDs in a dictionary (and /saids/ list)
"""
#print(saids, '<- old')
sub_account_ids, fids = {}, []
for firm_name, firm_re in firm_res.items():
    if firm_name in force_ids['saids']: # check overrides
        sub_account_ids[firm_name] = force_ids['saids'][firm_name]
        continue
    if len(managing_firm_ids[firm_name]) > 0: # no mids match
        sub_account_ids[firm_name] = []
        continue
    sub_account_ids[firm_name] = list(dfs['fund'][
            dfs['fund'].apply(lambda x: bool(firm_re.search(x.sub_account_name)), axis=1)
        ].sub_account_id.unique())
saids = list(map(int, chain(*sub_account_ids.values())))

In [13]:
all_ids = []
# Add managing firm IDs
for search_name, search_ids in managing_firm_ids.items():
    for x in managing_firm_ids[search_name]:
        all_ids.append(( search_name, x, pd.np.nan))
# Add sub account IDs
for search_name, search_ids in sub_account_ids.items():
    for x in sub_account_ids[search_name]:
        all_ids.append(( search_name, pd.np.nan, x))
#tmp = (pd.DataFrame(all_ids)
  #      .sort(columns='managing_firm_id')
#         .to_csv('/home/gaulinmp/Dropbox/Documents/School/Projects/'+
#                 'KBP_katrina_bond_to_private/Katrina_Bond_Priceimpact/data/matched_firms.csv'))
#pd.DataFrame(all_ids, columns=['name', 'managing_firm_id','sub_account_id'])#.sort(columns='managing_firm_id')

# Postgres livin
Pull postgres stuffs from things.

In [14]:
def open_pg_connection():
    """
    engine = sa.create_engine('postgresql+psycopg2:///kbd?host=/var/run/postgresql')
    metadata = sa.MetaData(engine)
    metadata.reflect(bind=engine)
    psql_tables = metadata.tables
    """
    global engine, metadata, psql_tables
    engine = sa.create_engine('postgresql+psycopg2:///kbd?host=/var/run/postgresql')
    metadata = sa.MetaData(engine)
    metadata.reflect(bind=engine)
    psql_tables = metadata.tables

def pg_to_df(query):
    """
    Executes sqlalchemy query on engine connection, and returns dataframe with labeled columns.
    If engine does not exist, creates default connection.
    """
    try:
        engine.name
    except (NameError, AttributeError):
        open_pg_connection()
    with engine.connect() as connection:
        results = (connection
                   .execution_options(stream_results=True)
                   .execute(query))
        dataframe = pd.DataFrame(iter(results))
        dataframe.columns = results.keys()
    return dataframe

def pg_head(query, N=5):
    """
    Executes sqlalchemy query on engine connection, and 
    returns first N lines in dataframe with labeled columns.
    If engine does not exist, creates default connection.
    """
    try:
        engine.name
    except (NameError, AttributeError):
        open_pg_connection()
    with engine.connect() as connection:
        results = (connection
                   .execution_options(stream_results=True)
                   .execute(query))
        dataframe = pd.DataFrame(results.fetchmany(N))
        dataframe.columns = results.keys()
    return dataframe

def kill_pg_connection():
    """Kills open connections."""
    global engine, metadata, psql_tables, query
    try: query.close()
    except: print("Didn't close query.")
    try: engine.dispose()
    except: print("Didn't dispose engine.")
    (engine, metadata, psql_tables, query) = [None]*4

In [16]:
open_pg_connection()
sql_mf = psql_tables['firm']
sql_sa = psql_tables['fund']
sql_holding = psql_tables['holding']
sql_issuers = psql_tables['issuers']
sql_securities = psql_tables['secmast']

# Link insurance companies to bonds
1. Get treated insurance companies *sub_account_class*
1. Get all companies of *sub_account_class*
1. Get master file of owned firms (treated and control)
1. Link to holdings
11. Add treated dummy
1. Get master list of treated and control firms
1. Get firm-quarter level bond data
11. Add treated dummy
11. Add *portion_treated* column
11. Add *portion_insurance* column

## TODO:
1. Ratio of EMAXX / Compustat Liabilities
1. Treated is all sub accounts under treated managing firms, control is all insurance sub-accounts

In [17]:
SA_CLASSES = "LIN PIN RIN INS".split(' ')
MF_CLASSES = "NO_IMD NO_ILF NO_IND IPC REI".split(' ')

In [18]:
# List of affected firms Sub-Account IDs
sq_firm_sa = (
    sa.select([sql_sa.c.managing_firm_id,
               sql_sa.c.sub_account_id,
               sql_sa.c.sub_account_class,])
      .where(sa.and_(sql_sa.c.sub_account_class.in_(SA_CLASSES),
                     sa.or_(sql_sa.c.managing_firm_id.in_(mids),
                            sql_sa.c.sub_account_id.in_(saids),),
                    sql_sa.c.year==2005,
                    sql_sa.c.quarter.in_((2, 3, ))))
      .distinct()
).alias('sq_firm_sa')

sq_sa_ids_treated = (
    sa.select([sq_firm_sa.c.sub_account_id,])
      .distinct()
).alias('sq_sa_ids_treated')

print("Treated Sub Account IDs:",len(pg_to_df(sq_sa_ids_treated).index))
pg_head(sq_firm_sa)

Treated Sub Account IDs: 141


,managing_firm_id,sub_account_id,sub_account_class
0,10032,22240,PIN
1,10032,38667,LIN
2,12834,39108,PIN
3,27633,20202,PIN
4,27633,22127,PIN


In [19]:
tmp = pg_to_df(
    sa.select([sq_firm_sa.c.managing_firm_id,])
      .distinct()
)
len(tmp.index)

17

In [20]:
tmp = pg_to_df(
  sa.select([sql_sa.c.managing_firm_id, sql_mf.c.managing_firm_type, sql_mf.c.managing_firm_name])
    .select_from(sa.join(sql_mf, sql_sa,
                         sa.and_(sql_mf.c.managing_firm_id==sql_sa.c.managing_firm_id,
                                 sql_mf.c.year==sql_sa.c.year,
                                 sql_mf.c.quarter==sql_sa.c.quarter,)))
    .where(sa.and_(sql_mf.c.managing_firm_type.in_(MF_CLASSES),
                   sql_sa.c.sub_account_class.in_(SA_CLASSES),
                   sql_sa.c.sub_account_id > 0,
                   sql_sa.c.managing_firm_id > 0))
    .distinct()
)
len(tmp.managing_firm_id.unique())

517

In [21]:
tmp.merge(dfs['firmtype'])[['managing_firm_type', 'managing_firm_type_name']].drop_duplicates()

,managing_firm_type,managing_firm_type_name
0,IPC,Insurance Co-Prop & Cas
932,REI,Reinsurance Company


In [22]:
print(sql_mf.c.keys())

['managing_firm_id', 'year', 'quarter', 'managing_firm_name', 'managing_firm_name_sort', 'managing_firm_type', 'managing_firm_country_code', 'managing_firm_total_par_held', 'managing_firm_number_issues_held']


### 2. All firms with correct SA class 

In [23]:
sq_sa_ids_all = (
  sa.select([sql_sa.c.sub_account_id])
    .select_from(sa.join(sql_mf, sql_sa,
                         sa.and_(sql_mf.c.managing_firm_id==sql_sa.c.managing_firm_id,
                                 sql_mf.c.year==sql_sa.c.year,
                                 sql_mf.c.quarter==sql_sa.c.quarter,)))
    .where(sa.and_(sql_mf.c.managing_firm_type.in_(MF_CLASSES),
                   sql_sa.c.sub_account_class.in_(SA_CLASSES),
                   sql_sa.c.sub_account_id > 0,
                   sql_sa.c.managing_firm_id > 0))
    .distinct()
).alias('sq_sa_ids_all')

print("Total Sub Account IDs:",len(pg_to_df(sq_sa_ids_all).index))

Total Sub Account IDs: 1117


### 3. All owned firms CUSIP6s

In [24]:
sq_cusip_all = (
  sa.select([sql_holding.c.cusip6,])
    .where(sql_holding.c.sub_account_id.in_(sq_sa_ids_all)) 
    .distinct()
).alias('sq_cusip_all')

print("Total CUSIP6s:",len(pg_to_df(sq_cusip_all).index))

Total CUSIP6s: 14884


### 4. Link all SA IDs to Holding
**`sq_holding_all`** will have cusip6, sub_account, year, quarter level obervations.
Group on cusip6, year, quarter to match to Compustat.

In [25]:
sq_holding_all = (
  sa.select(sql_holding.c + [
             sa.case([(sql_holding.c.sub_account_id.in_(sq_sa_ids_treated), 1)],
                     else_=0).label('treated'),
             sa.case([(sql_holding.c.sub_account_id.in_(sq_sa_ids_all), 1)],
                     else_=0).label('insurance'),])
    #.select_from(sa.join(sql_holding, sq_sa_ids, 
    #                     sql_holding.c.sub_account_id == sq_sa_ids.c.sub_account_id, isouter=True))
    .where(sa.and_(sql_holding.c.cusip6.in_(sq_cusip_all),
                   sa.between(sql_holding.c.year, 2004, 2007),))
).alias('sq_holding_all')

### 5. Link all SA IDs to Holding
**`sq_holding_all_collapsed`** will have cusip6, year, quarter level obervations.
Importantly, columns **`par_amount`** and **`par_amount_treated`** will be the amount of outstanding bonds held by errbudy and affected insurance companies respectively.

In [26]:
sq_holding_all_collapsed = (
  sa.select([sq_holding_all.c.cusip6,
             sq_holding_all.c.year,
             sq_holding_all.c.quarter,
             sa.func.sum(sq_holding_all.c.par_amount).label('par'),
             sa.func.sum(sq_holding_all.c.par_amount_change).label('par_change'),
             sa.func.sum(sq_holding_all.c.par_amount 
                         * sq_holding_all.c.insurance).label('par_insurance'),
             sa.func.sum(sq_holding_all.c.par_amount_change 
                         * sq_holding_all.c.insurance).label('par_change_insurance'),
             sa.func.sum(sq_holding_all.c.par_amount 
                         * sq_holding_all.c.treated).label('par_treated'),
             sa.func.sum(sq_holding_all.c.par_amount_change 
                         * sq_holding_all.c.treated).label('par_change_treated'),])
    .distinct()
    .group_by(sq_holding_all.c.cusip6,
              sq_holding_all.c.year,
              sq_holding_all.c.quarter,)
    .order_by(sq_holding_all.c.year,
              sq_holding_all.c.quarter,
              sq_holding_all.c.cusip6,)
).alias('sq_holding_all_collapsed')

### 6. Link in firm names for matching.

In [27]:
sq_holding_final = (
  sa.select(sq_holding_all_collapsed.c+
            [sql_issuers.c.name,
             sql_issuers.c.ticker,
             sql_issuers.c.country,])
    .select_from(sa.join(sq_holding_all_collapsed,  # left
                         sql_issuers,  # right
                         sa.and_(sq_holding_all_collapsed.c.cusip6 == sql_issuers.c.cusip6,
                                 sq_holding_all_collapsed.c.year == sql_issuers.c.year,
                                 sq_holding_all_collapsed.c.quarter == sql_issuers.c.quarter,)))
    .distinct(sq_holding_all_collapsed.c.year,
              sq_holding_all_collapsed.c.quarter,
              sq_holding_all_collapsed.c.cusip6,
              sq_holding_all_collapsed.c.par_treated,)
).alias('sq_holding_final')

### 7. Export to csv file

In [28]:
df_emaxx = pg_to_df(sq_holding_final)
for v in [x for x in df_emaxx.columns if x[:3]=='par']:
    df_emaxx[v] = df_emaxx[v].apply(int)
df_emaxx = add_column(df_emaxx, 'date', lambda x: dt.datetime(x['year'], x['quarter']*3, 30))
df_emaxx = add_column(df_emaxx, 'iown', lambda x: x['par_insurance']/x['par'] if x['par'] > 0 else 0)
df_emaxx = add_column(df_emaxx, 'iown_treated', lambda x: x['par_treated']/x['par'] if x['par'] > 0 else 0)

In [29]:
#df_emaxx.sort(columns='cusip6,year,quarter'.split(',')
#             ).to_csv(os.path.join(ROOTDIR,'all_out.tsv'), index=False)

In [32]:
print(len(df_emaxx[(df_emaxx.year==2005) & (df_emaxx.quarter == 2) & (df_emaxx.par_treated > 0)].cusip6.unique()))
df_emaxx.dtypes

2996


cusip6                          object
year                             int64
quarter                          int64
par                              int64
par_change                       int64
par_insurance                    int64
par_change_insurance             int64
par_treated                      int64
par_change_treated               int64
name                            object
ticker                          object
country                         object
date                    datetime64[ns]
iown                           float64
iown_treated                   float64
dtype: object

In [31]:
df_emaxx[(df_emaxx.year==2005) & (df_emaxx.quarter == 2) & (df_emaxx.par_insurance > 0)].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
year,4777,2005.000,0.000,2.005e+03,2005.000,2005.000,2005.000,2005
quarter,4777,2.000,0.000,2.000e+00,2.000,2.000,2.000,2
par,4777,383244.593,821547.259,7.000e+00,37771.000,141252.000,379128.000,15801676
par_change,4777,18571.103,87397.604,-8.807e+05,-565.000,0.000,12506.000,1582802
par_insurance,4777,26857.880,64761.140,1.000e+00,1400.000,7205.000,25000.000,1251717
par_change_insurance,4777,874.581,8946.499,-8.811e+04,0.000,0.000,55.000,300050
par_treated,4777,14995.928,33432.469,0.000e+00,0.000,2000.000,16400.000,510564
par_change_treated,4777,451.616,6039.647,-9.500e+04,0.000,0.000,0.000,119968
iown,4777,0.195,0.293,3.380e-06,0.027,0.069,0.186,1
iown_treated,4777,0.071,0.162,0.000e+00,0.000,0.012,0.066,1


## Par Amounts aggregated from HOLDINGS database

In [ ]:
tmp = df_emaxx.pivot_table(index=['date',], aggfunc='mean')
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        axtmp = ax1.twinx() if i else ax1
        axtmp.plot(tmp.index, 
                 tmp.iown_treated if i else tmp.iown,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    axtmp.legend(loc=0)
    fig.suptitle('Mean Par Amount Outstanding')

In [ ]:
tmp = df_emaxx.pivot_table(index=['date',], aggfunc='mean')
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        axtmp = ax1.twinx() if i else ax1
        axtmp.plot(tmp.index, 
                 tmp.par_treated if i else tmp.par-tmp.par_treated,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    axtmp.legend(loc=0)
    fig.suptitle('Mean Par Amount Outstanding')

## Change in Par Amounts aggregated from HOLDINGS database

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        tmp_var = tmp.par_treated if i else tmp.par
        ax1.plot(tmp.index, 
                 tmp_var/tmp_var.shift(),
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend(loc=0)
    fig.suptitle('Change in Mean Par Amount Outstanding')

In [ ]:
tmp = df_emaxx.copy().sort(columns=['cusip6', 'date'])

In [ ]:
tmp['lag_par'] = tmp.groupby('cusip6').par.shift()
tmp['lag_par_treated'] = tmp.groupby('cusip6').par_treated.shift()
tmp['lag_iown'] = tmp.groupby('cusip6').iown.shift()
tmp['lag_iown_treated'] = tmp.groupby('cusip6').iown_treated.shift()
tmp['iown_change'] = tmp.iown - tmp.lag_iown
tmp['iown_change_treated'] = tmp.iown_treated - tmp.lag_iown_treated

In [ ]:
formula = "iown_change ~ lag_iown"
model = sm.OLS.from_formula(formula, tmp[(tmp.year == 2005) & (tmp.quarter == 2)], missing='drop')
fit = model.fit()
summary = str(fit.summary2())
print(summary)
formula = "iown_change ~ lag_iown"
model = sm.OLS.from_formula(formula, tmp[(tmp.year == 2005) & (tmp.quarter == 3)], missing='drop')
fit = model.fit()
summary = str(fit.summary2())
print()
print(summary)

In [ ]:
formula = "iown_change_treated ~ lag_iown_treated"
model = sm.OLS.from_formula(formula, tmp[(tmp.year == 2005) & (tmp.quarter == 2)], missing='drop')
fit = model.fit()
summary = str(fit.summary2())
#tmp.loc[:,'reg_resid'] = fit.resid
print(summary)
formula = "iown_change_treated ~ lag_iown_treated"
model = sm.OLS.from_formula(formula, tmp[(tmp.year == 2005) & (tmp.quarter == 3)], missing='drop')
fit = model.fit()
summary = str(fit.summary2())
#tmp.loc[:,'reg_resid'] = fit.resid
print(summary)

In [ ]:
print("E[treated/par]={:0.4f}\nE[treated/par|treated>0]={:0.4f}".format(
(df_emaxx[df_emaxx.par > 0].par_treated/(df_emaxx[df_emaxx.par > 0].par)).mean(), \
(df_emaxx[df_emaxx.par_treated > 0].par_treated/(df_emaxx[df_emaxx.par_treated > 0].par)).mean()))

# Sub-account level holding changes

In [ ]:
sq_sa_holding = (
  sa.select(sql_sa.c +
            [sa.case([(sql_sa.c.sub_account_id.in_(sq_sa_ids_treated), 1)],
                     else_=0).label('treated'),])
    .where(sa.and_(sql_sa.c.sub_account_class.in_(SA_CLASSES),
                   sql_sa.c.sub_account_id > 0,
                   sql_sa.c.sub_account_country == "USA"))
    .distinct()
).alias('sq_sa_holding')

In [ ]:
df_emaxx_sa = pg_to_df(sq_sa_holding)
df_emaxx_sa = add_column(df_emaxx_sa, 'date', lambda x: dt.datetime(x['year'], x['quarter']*3, 30))

In [ ]:
tmp = df_emaxx_sa.pivot_table(index=['date','treated'], aggfunc='sum')
tmp.head(4).transpose()

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        axtmp = ax1.twinx() if i else ax1
        axtmp.plot(tmp.xs(i, level='treated').index, 
                 tmp.xs(i, level='treated').total_par_amt_held,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Total Par Amount Outstanding')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        tmpi = tmp.xs(i, level='treated')
        ax1.plot(tmpi.index,
                 tmpi.total_par_amt_held/tmpi.total_par_amt_held.shift(),
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Total Par Amount Growth [t/(t-1)]')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        axtmp = ax1.twinx() if i else ax1
        axtmp.plot(tmp.xs(i, level='treated').index, 
                 tmp.xs(i, level='treated').total_number_bonds_held,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Total Number Outstanding')

In [ ]:
tmp = df_emaxx_sa.pivot_table(index=['date','treated'], aggfunc='mean')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        axtmp = ax1.twinx() if i else ax1
        axtmp.plot(tmp.xs(i, level='treated').index, 
                 tmp.xs(i, level='treated').total_par_amt_held,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Average Par Amount Outstanding')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        tmpi = tmp.xs(i, level='treated')
        ax1.plot(tmpi.index,
                 tmpi.total_par_amt_held/tmpi.total_par_amt_held.shift(),
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Mean Par Amount Growth [t/(t-1)]')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        axtmp = ax1.twinx() if i else ax1
        axtmp.plot(tmp.xs(i, level='treated').index, 
                 tmp.xs(i, level='treated').total_number_bonds_held,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Total Number Outstanding')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        tmpi = tmp.xs(i, level='treated')
        ax1.plot(tmpi.index,
                 tmpi.total_number_bonds_held/tmpi.total_number_bonds_held.shift(),
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Mean Number Growth [t/(t-1)]')

# Random descriptives

In [ ]:
print("<b>EMAXX sample</b>")
df_emaxx.head(2)

In [ ]:
df_comp = dfs['compcusip'][dfs['compcusip'].year.notnull()]
df_comp.columns = [x.lower() for x in df_comp.columns]
HTML("<b>Compustat sample</b>")
df_comp.head(2)

In [ ]:
df_b = pd.merge(
    df_emaxx[df_emaxx.country == "USA"],
    df_comp[df_comp.fic == "USA"],
    how='outer')
df_b['emaxx'] = df_b.par.notnull() + 0
df_b['comp'] = df_b.permno.notnull() + 0
df_b['both'] = (df_b.emaxx == df_b.comp) + 0

In [ ]:
df_b[(df_b.year==2005)&(df_b.quarter==3)].describe().transpose()

In [ ]:
df_b[(df_b.year==2005)&(df_b.quarter==3)&(df_b.both==1)].describe().transpose()

### Previous run results

In [ ]:
df_b[(df_b.year==2005)&(df_b.quarter==3)].describe().transpose()

In [ ]:
df_b[(df_b.year==2005)&(df_b.quarter==3)&(df_b.both==1)].describe().transpose()

In [ ]:
df_emaxx.dtypes

In [ ]:
query = (sa.select(columns=[sql_holding.c.year, 
                            sql_holding.c.quarter, 
                            sa.func.count().label('num')]
                   , limit=None)
           .where(sa.and_(sql_holding.c.managing_firm_id.in_(mids),
                          sql_holding.c.year.in_([2004,2005,2006,2007])))
           .group_by(sql_holding.c.year, sql_holding.c.quarter)
           .order_by(sql_holding.c.year, sql_holding.c.quarter)
        )
tmp = pg_to_df(query)
tmp = add_column(tmp, 'date', lambda x: dt.datetime(x['year'], x['quarter']*3, 30))
zmr = 300
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    ax1.plot(tmp.date, tmp.num, 'bo-')
    ax1.axvline(color='black', x=KATRINA_DATE)
    fig.suptitle('Number of Bonds Outstanding (MgrFirm)')

In [ ]:
# List of firms with type found in searched firms
sq_firm_types = (
      sa.select([sql_mf.c.managing_firm_type])
        .where(sql_mf.c.managing_firm_id.in_(mids))
        .distinct()
    ).alias('sq_firm_types')

# List of all firms, control and treated of right type
sq_sample_firms = (
      sa.select([sql_mf.c.managing_firm_id], distinct=True)
        .where(sa.and_(
            sql_mf.c.managing_firm_type.in_(sq_firm_types)
            ,sql_mf.c.managing_firm_id.notin_(mids)
        ))
    ).alias('sq_sample_firms')

# Count holdings for sample firms.
query = (sa.select([sql_holding.c.year, 
                    sql_holding.c.quarter, 
                    sa.func.count().label('num')], limit=500)
           .distinct()
           .where(sa.and_(sql_holding.c.managing_firm_id.in_(sq_sample_firms),
                          sql_holding.c.year.in_([2004,2005,2006,2007])))
           .group_by(sql_holding.c.year, sql_holding.c.quarter)
           .order_by(sql_holding.c.year, sql_holding.c.quarter)
        )

tmp = pg_to_df(query)
tmp = add_column(tmp, 'date', lambda x: dt.datetime(x['year'], x['quarter']*3, 30))
zmr = 300
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    ax1.plot(tmp.date, tmp.num, 'bo-')
    ax1.axvline(color='black', x=KATRINA_DATE)
    fig.suptitle('Number of Bonds Outstanding (MgrFirm)')

In [ ]:
query = (sa.select(columns=[sql_holding.c.year, 
                            sql_holding.c.quarter, 
                            sa.func.count().label('num')]
                   , limit=None)
           .where(sa.and_(sql_holding.c.sub_account_id.in_(saids),
                          sql_holding.c.year.in_([2004,2005,2006])))
           .group_by(sql_holding.c.year, sql_holding.c.quarter)
           .order_by(sql_holding.c.year, sql_holding.c.quarter)
        )
tmp = pg_to_df(query)
tmp = add_column(tmp, 'date', lambda x: dt.datetime(x['year'], x['quarter']*3, 30))
zmr = 300
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    ax1.plot(tmp.date, tmp.num, 'bo-')
    ax1.axvline(color='black', x=KATRINA_DATE)
    fig.suptitle('Number of Bonds Outstanding (SubAcct)')

In [ ]:
stmnt = (sa.select(columns=sql_holding.c, limit=None)
           .where(sql_holding.c.managing_firm_id.in_(mids)) 
        ).alias('stmnt')
query = (sa.select(columns=[stmnt.c.year,
                            stmnt.c.quarter,
                            sa.func.sum(stmnt.c.par_amount_change).label('ave_par_amount_change')]
                   , limit=None)
            .group_by(stmnt.c.year, stmnt.c.quarter)
            .order_by(stmnt.c.year, stmnt.c.quarter)
        )
dataframe = pg_to_df(query)
dataframe.head()

In [ ]:
stmnt = (sa.select(columns=sql_holding.c, limit=None)
           .where(sql_holding.c.managing_firm_id.notin_(mids)) 
        ).alias('stmnt')
query = (sa.select(columns=[stmnt.c.year,
                            stmnt.c.quarter,
                            sa.func.sum(stmnt.c.par_amount_change).label('ave_par_amount_change')]
                   , limit=None)
            .group_by(stmnt.c.year, stmnt.c.quarter)
            .order_by(stmnt.c.year, stmnt.c.quarter)
        )
with engine.connect() as connection:
    results = (connection
               .execution_options(stream_results=True)
               .execute(query))
    dataframe2 = pd.DataFrame(iter(results))
    dataframe2.columns = results.keys()
    dataframe2.sort(columns=['year','quarter'])
dataframe2.head()

In [ ]:
lookup = 'forecasts'
zmr = 300
tmp = [a*10 + b for a,b in dataframe['year,quarter'.split(',')].values]
if True:
    fig, ax1 = plt.subplots(figsize=(15,4))
    ax2 = ax1.twinx()
    ax1.plot(tmp, dataframe.ave_par_amount_change/dataframe.ave_par_amount_change.max()*100, )
    ax2.plot(tmp, dataframe2.ave_par_amount_change/dataframe2.ave_par_amount_change.max()*100, 'r--')
    ax1.axvline(color='black', x=20052)

In [ ]:
query = (sa.select(columns=[sa.func.sum(sql_holding.c.par_amount_change)]
                   , limit=None)
#        .where(sql_holding.c.cusip6=='001957')
#        .where(sql_holding.c.cusip78=='AV')
        .where(sql_holding.c.managing_firm_id.in_(mids))
        .group_by(sql_holding.c.year, sql_holding.c.quarter)
        )
with engine.connect() as connection:
    results = (connection
               .execution_options(stream_results=True)
               .execute(query))
    tmpdf = pd.DataFrame(iter(results))
    tmpdf.columns = results.keys()
print(len(tmpdf.index))
tmpdf.head()

In [ ]:
tmpdf['year,quarter,sub_account_id,par_amount,report_date,par_amount_change,acquisition_book_value'.\
      split(',')].sort('sub_account_id').dtypes

In [ ]:
#kill_pg_connection()

# Compustat Data

In [ ]:
df_comp = pd.read_csv('../../comp_data.tab', sep='\t', parse_dates=['date_start', 'date_end', 'date'])
df_comp.columns = [x.lower() for x in df_comp.columns]
dfc_k = (df_comp.year == KATRINA_DATE.year) & (df_comp.quarter == KATRINA_QUARTER)
dfc_prek = (df_comp.year == KATRINA_DATE.year) & (df_comp.quarter == KATRINA_QUARTER-1)

treated_series = add_column(df_comp[dfc_prek][['gvkey', 'par_treated']], 
                            'treated',
                            lambda x: int(x.par_treated > 0))
df_comp = pd.merge(df_comp, 
                   treated_series[['gvkey', 'treated']], 
                   on='gvkey')
#print(df_comp.head(1).dtypes)

In [ ]:
cusip6_good = list(df_comp.cusip6.unique())

In [ ]:
deets = \
df_comp[['date', 'atq', 'ltq', 'niq', 'req', 'par_treated_bool',
         'par', 'par_change', 'par_treated', 'par_change_treated',
         'how_treated', 'treated_rank', 'katrina_rank']
       ].groupby(['date', 'treated_rank']).describe()

In [ ]:
if plots:
    for var_to_plot in ['atq', 'ltq', 'niq', 'req']:
        fig, ax1 = plt.subplots(figsize=(15,4))
        summ_stat =  'mean'
        for i in deets.index.get_level_values(1).unique():
            tmpi = deets.xs(i, level='treated_rank').xs(summ_stat, level=1)
            ax1.plot(tmpi.index,
                     tmpi[var_to_plot],
                     label="Rank {}".format(i))
        ax1.axvline(color='black', x=KATRINA_DATE)
        ax1.legend()
        fig.suptitle('Ranks of {}'.format(var_to_plot))

In [ ]:
deets = df_comp[
    ['date', 'atq', 'ltq', 'niq', 'req',
     'par', 'par_change', 'par_treated', 'par_change_treated',
     'how_treated', 'treated_rank', 'katrina_rank']
    ].groupby(['date', 'katrina_rank']).describe()
if plots:
    for var_to_plot in ['atq', 'ltq', 'niq', 'req']:
        fig, ax1 = plt.subplots(figsize=(15,4))
        summ_stat =  'mean'
        for i in deets.index.get_level_values(1).unique():
            tmpi = deets.xs(i, level='katrina_rank').xs(summ_stat, level=1)
            ax1.plot(tmpi.index,
                     tmpi[var_to_plot],
                     label="Rank {}".format(i))
        ax1.axvline(color='black', x=KATRINA_DATE)
        ax1.legend()
        fig.suptitle('Ranks of {}'.format(var_to_plot))

In [ ]:
deets = df_comp[df_comp.katrina_rank.notnull()][
    ['date', 'atq', 'ltq', 'niq', 'req', 'par_treated_bool',
     'par', 'par_change', 'par_treated', 'par_change_treated',
     'how_treated', 'treated_rank', 'katrina_rank']
    ].groupby(['date', 'par_treated_bool']).describe()
if plots:
    for var_to_plot in ['atq', 'ltq', 'niq', 'req', 'par', 'par_treated',
                        'par_change', 'par_change_treated']:
        fig, ax1 = plt.subplots(figsize=(8,4))
        summ_stat =  'mean'
        for i in deets.index.get_level_values(1).unique():
            tmpi = deets.xs(i, level='par_treated_bool').xs(summ_stat, level=1)
            ax1.plot(tmpi.index,
                     tmpi[var_to_plot],
                     'bo-' if i else 'ro--',
                     label="Treated" if i else "Control")
        ax1.axvline(color='black', x=KATRINA_DATE)
        ax1.legend()
        fig.suptitle('Mean {}'.format(var_to_plot))

In [ ]:
deets = df_comp[df_comp.katrina_rank.notnull()][
    ['date', 'atq', 'ltq', 'niq', 'req', 'par_treated_bool',
     'par', 'par_change', 'par_treated', 'par_change_treated',
     'how_treated', 'treated_rank', 'katrina_rank']
    ].groupby(['date', 'par_treated_bool']).describe()
if plots:
    for var_to_plot in ['atq', 'ltq', 'niq', 'req', 'par', 'par_treated',
                        'par_change', 'par_change_treated']:
        fig, ax1 = plt.subplots(figsize=(8,4))
        summ_stat =  '50%'
        for i in deets.index.get_level_values(1).unique():
            tmpi = deets.xs(i, level='par_treated_bool').xs(summ_stat, level=1)
            ax1.plot(tmpi.index,
                     tmpi[var_to_plot],
                     'bo-' if i else 'ro--',
                     label="Treated" if i else "Control")
        ax1.axvline(color='black', x=KATRINA_DATE)
        ax1.legend()
        fig.suptitle('Mean {}'.format(var_to_plot))

# TODO:
1. Ratio of EMAXX / Compustat Liabilities
1. Treated is all sub accounts under treated managing firms, control is all insurance sub-accounts

In [ ]:
tmp = df_comp[df_comp.gvkey.notnull()].copy()
tmp = add_column(tmp, 'debt_ratio_all',
                 lambda x: x.par / x.dlttq / 1000 if x.dlttq > 0 else 0)
tmp = add_column(tmp, 'debt_ratio',
                 lambda x: x.par / x.dlttq / 1000 if x.dlttq > x.par/1000 else 1)
print(tmp.columns)

In [ ]:
tmp.describe().transpose()

In [ ]:
tmpp = tmp.pivot_table(index=['date','treated'], aggfunc='mean')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        tmpi = tmpp.xs(i, level='treated')
        ax1.plot(tmpi.index,
                 tmpi.debt_ratio,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Average Bond/Debt ratio')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        tmpi = tmpp.xs(i, level='treated')
        ax1.plot(tmpi.index,
                 tmpi.dlttq,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Average Debt outstanding')

In [ ]:
if plots:
    fig, ax1 = plt.subplots(figsize=(15,4))
    for i in (0,1):
        tmpi = tmpp.xs(i, level='treated')
        ax1.plot(tmpi.index,
                 tmpi.par/1000,
                 'bo-' if i else 'ro--',
                 label="Treated" if i else "Control")
    ax1.axvline(color='black', x=KATRINA_DATE)
    ax1.legend()
    fig.suptitle('Average Par Amount')